### Callin Switzer

1 Nov 2016

-Read in all the Kalmia polygon images (from R KalmiaVisualizePolygons_V2.R)

-Weight pixels with increasing values as they get closer to zero

-Resample all of the images

-Stack resampled images

-Visualize a heatmap

-Save heatmap

In [463]:
import cv2

import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

In [411]:
%qtconsole

In [3]:
# list files in directory

In [89]:
os.chdir("/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaExamples/KalmiaManualTrig_VidsToProcess/")

In [319]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [556]:
# visualize individual polygons with weighting

kk = 0
fpth = photoList[kk]
img = cv2.imread(fpth,0)
img = (img < 255) * 1.0

# these are the points of interest
intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])

#img[:, 1000]= 0.9
#img[1694, :] = 0.9
plt.clf()
io.imshow(img, cmap = 'gray')
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])



# make multiplication matrix
mask = np.ones((2000, 2000))

# make points exponentially decreasing 
# this makes the center probability  == 10 times the outside probability
row1 = (10.0**((np.hstack([range(0, 1000), range(999, -1, -1)]))/231.0))

# normalize
row1 = row1 / np.max(row1)

# make a matrix to multiply by
mask2 = mask * (row1 )
io.imshow(mask2 * img, cmap = 'gray')
print row1[768], row1[1000], row1[1231]
print  row1[1000] / float(row1[768])

0.1 1.0 0.1
10.0


In [561]:
# stack all polygons and visualize without weighting
stackedImg = np.ones((2000, 2000)) * 0.0



for ii in range(len(photoList)):
    # read in image
    
    fpth = photoList[ii]
    img = cv2.imread(fpth,0)
    img = (img < 255) * 1.0
    if ii == 0:
        img = cv2.flip(img, 1)
    img2 = mask * img
    stackedImg = img2 + stackedImg
    
plt.clf()
io.imshow(stackedImg, cmap = 'YlOrRd')

In [566]:
# resample stacked images without weighting
def resampleImgsNoWeight():
    # stack all polygons and visualize
    stackedImg = np.ones((2000, 2000)) * 0.0

    # make multiplication matrix
    mask = np.ones((2000, 2000))

    photoSamp = np.random.choice(range(len(photoList)), size=len(photoList), replace=True, p=None)


    for ii in photoSamp:
        # read in image
        fpth = photoList[ii]
        img = cv2.imread(fpth,0)
        img = (img < 255) * 1.0



        #flip coin for whether or not to flip image
        if fpth == '20160615_121016.png':
            img = cv2.flip(img, 1)

        img2 = mask * img

        # stack images
        stackedImg = img2 + stackedImg
    
    return stackedImg

In [567]:
img1 = resampleImgsNoWeight()

In [568]:
plt.clf()
io.imshow(img1)

In [648]:
stt = time.time()
imStt = np.array([resampleImgsNoWeight() for ii in range(100)])
print time.time() - stt

226.997791052


In [649]:
# sum up arrays to make total bootstrap resample 
imSum = imStt.sum(axis = 0)

# normalize
imSum = imSum / np.max(imSum)

plt.clf()
io.imshow(imSum, cmap = 'magma')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])

In [654]:
os.path.abspath(photoList[kk])

'/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaExamples/KalmiaManualTrig_VidsToProcess/20160615_121016.png'

In [653]:
# add  blur
plt.clf()
kernel = np.ones((100,100),np.float32)/25
imSum2 = np.array(cv2.filter2D(imSum.copy(),-1,kernel))

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')

fig, ax = plt.subplots()
im = ax.imshow(imSum2, cmap = 'Purples')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')

# add contour lines
ax.contour(imSum2, levels = range(10, 100, 20))
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('SingleAntherHeatmap_Purples.png')

In [597]:
plt.clf()
io.imshow(np.array(imSum2), cmap = 'magma')

In [610]:
%qtconsole

In [449]:
plt.clf()
plt.hist(np.ndarray.flatten(stackedImg))
plt.ylim([0, 500])

(0, 500)

In [444]:
# stack all polygons and visualize, with weighting
stackedImg = np.ones((2000, 2000)) * 0.0

# make multiplication matrix
mask = np.ones((2000, 2000))

# make points exponentially decreasing 
# this makes the center probability  == 10 times the outside probability
row1 = (10.0**((np.hstack([range(0, 1000), range(999, -1, -1)]))/231.0))

# normalize
row1 = row1 / np.max(row1)
mask2 = mask * (row1)


for ii in range(len(photoList)):
    # read in image
    fpth = photoList[ii]
    img = cv2.imread(fpth,0)
    img = (img < 255) * 1.0
    img2 = mask2 * img
    stackedImg = img2 + stackedImg
    
plt.clf()
io.imshow(stackedImg)

In [328]:
# resample polygons and flip random ones

io.imshow(cv2.flip(img, 1))

In [450]:
def resampleImgs():
    # stack all polygons and visualize
    stackedImg = np.ones((2000, 2000)) * 0.0

    # make multiplication matrix
    mask = np.ones((2000, 2000))

    # make points exponentially decreasing 
    # this makes the center probability  == 10 times the outside probability
    row1 = (10.0**((np.hstack([range(0, 1000), range(999, -1, -1)]))/231.0))

    # normalize
    row1 = row1 / np.max(row1)
    mask2 = mask * (row1)




    #sample photolist
    photoSamp = np.random.choice(range(len(photoList)), size=len(photoList), replace=True, p=None)


    for ii in photoSamp:
        # read in image
        fpth = photoList[ii]
        img = cv2.imread(fpth,0)
        img = (img < 255) * 1.0



        #flip coin for whether or not to flip image
        flip = np.random.choice([True, False], size=1, replace=True, p=None)
        if flip:
            img = cv2.flip(img, 1)

        img2 = mask2 * img

        # stack images
        stackedImg = img2 + stackedImg
    
    return stackedImg

In [454]:
imm = resampleImgs()
plt.clf()
plt.hist(np.ndarray.flatten(imm))
plt.ylim([0, 5000])

(0, 5000)

In [455]:
plt.clf()
io.imshow(imm)

In [629]:
stt = time.time()
imStt = np.array([resampleImgs() for ii in range(10)])
print time.time() - stt

27.2174310684


In [630]:
# sum up arrays to make total bootstrap resample 
imSum = imStt.sum(axis = 0)

# normalize
imSum = imSum / np.max(imSum)

plt.clf()
io.imshow(imSum, cmap = 'magma')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

(1700, 700)

In [496]:
plt.clf()
io.imshow(imSum, cmap = 'YlOrRd')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap_Yellow.png')

In [495]:
plt.clf()
io.imshow(imSum, cmap = 'Purples')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap_Purples.png')

In [499]:
plt.clf()
io.imshow(imSum, cmap = 'hot')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap_Hot.png')

In [469]:
plt.clf()
io.imshow(imSum, cmap = 'magma')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap.png')

(2000, 2000)

In [507]:
plt.clf()
io.imshow(imSum, cmap = 'gray_r')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
#plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
#plt.axis('off')

#plt.gca().set_position([0,0,1,1])
#plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

#plt.savefig('PollenHeatmap_BW.png')

In [471]:
plt.clf()
plt.hist(np.ndarray.flatten(imSum), bins = 100)
plt.ylim([0, 5000])

(0, 5000)

In [646]:
# plot with colorbar
# add blur and contour lines
# 

# sum up arrays to make total bootstrap resample 
imSum = imStt.sum(axis = 0)

# normalize
imSum = imSum / np.max(imSum) *100


kernel = np.ones((50,50),np.float32)/25
imSum2 = np.array(cv2.filter2D(imSum.copy(),-1,kernel))
# normalize
imSum2 = imSum2 / np.max(imSum2) * 100

plt.close('all')

fig, ax = plt.subplots()
im = ax.imshow(imSum2, cmap = 'gray_r')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1400])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])
ax.contour(imSum2, levels = range(10, 100, 20), cmap = 'Reds_r')

plt.show()
plt.savefig('PollenHeatmap_BW_colorbar_Blur.png')

In [519]:
data = np.arange(100, 0, -1).reshape(10, 10)

fig, ax = plt.subplots()
#cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])

im = ax.imshow(data, cmap='gist_earth')
fig.colorbar(im, cax=cax, orientation='horizontal')
plt.show()